In [4]:
import open3d as o3d
import torch
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
from plane_geometry import PlaneGeometry, orthonormal_basis_from_xy

In [6]:
points = torch.randn(1000, 3)
model = PlaneGeometry(10)
model.initialize(points, 20)
planes_points, planes_idx = model.sample_planes_points(2000)


In [7]:
r:float=2
colour = np.array([0.75,0.25,0.25])
alpha:float=0.5
objs = []
# point cloud
o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
o3d_pcd.paint_uniform_color(colour)

objs.append(o3d_pcd)

In [16]:
# plane
center = model.center.detach().cpu().numpy()
xyz = orthonormal_basis_from_xy(model.xy.detach()).detach().cpu().numpy()
wh = model.wh.detach().cpu().numpy()

colors = np.random.rand(model.n_plane, 3)
for i in range(model.n_plane):
    c = center[i]
    x, y = xyz[i,:,0], xyz[i,:,1]
    x_s, y_s = x*(wh[i, 0]/2), y*(wh[i, 1]/2)
    points = np.stack([c+x_s+y_s, c-x_s+y_s, c-x_s-y_s, c+x_s-y_s])
    lines = np.array([[0, 1], [1, 2], [2, 3], [3, 0]])
    plane = o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(points),
                                 lines=o3d.utility.Vector2iVector(lines))
    # plane.paint_uniform_color(colour)
    objs.append(plane)

In [17]:
o3d.visualization.draw_plotly(objs, mesh_show_wireframe=True,  up=[0, 0, 1], front=[1, 1, 1], lookat=[0.0, 0.0, 0.0], zoom=0.5)